In [1]:
import requests
import json
import time
import signal
import sys
import os
import toml


class StableUSDCPoolCollector:
    def __init__(self, results_file='stable_usdc_pools.toml'):
        self.results_file = results_file
        self.stable_usdc_pools = []
        self.pool_ids = set()
        self.interrupted = False
        self.setup_signal_handler()
        self.load_previous_results()
    
    def setup_signal_handler(self):
        signal.signal(signal.SIGINT, self.handle_interrupt)
    
    def handle_interrupt(self, signum, frame):
        print(f"\nInterrupted! Saving {len(self.stable_usdc_pools)} pools...")
        self.save_results()
        print(f"Results saved to {self.results_file}")
        self.display_results()
        sys.exit(0)
    
    def load_previous_results(self):
        if os.path.exists(self.results_file):
            try:
                with open(self.results_file, 'r') as f:
                    data = toml.load(f)
                    self.stable_usdc_pools = data.get('pools', [])
                    
                    self.pool_ids = set()
                    for pool in self.stable_usdc_pools:
                        pool_id = pool.get('pool_id')
                        if pool_id:
                            self.pool_ids.add(pool_id)
                    
                    count = len(self.stable_usdc_pools)
                    if count > 0:
                        print(f"Loaded {count} pools from previous run")
            except Exception:
                pass
    
    def save_results(self):
        sanctum_lst_list = []
        processed_mints = set()
        sol_mint = 'So11111111111111111111111111111111111111112'
        
        for pool in self.stable_usdc_pools:
            token_mints = pool.get('pool_token_mints', [])
            pool_address = pool.get('pool_address', '')
            pool_name = pool.get('pool_name', '')
            
            # Create entry for each token mint in the pool
            for mint in token_mints:
                if not mint or mint == sol_mint or mint in processed_mints:
                    continue
                
                processed_mints.add(mint)
                
                # Try to extract symbol from pool name
                symbol = ''
                if pool_name and '-' in pool_name:
                    tokens = pool_name.split('-')
                    # Find which token this mint represents
                    if len(tokens) >= 2:
                        usdc_addr = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
                        if mint != usdc_addr:
                            symbol = tokens[0] if tokens[0] != 'USDC' else tokens[1]
                        else:
                            symbol = 'USDC'
                
                token_prog = 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA'
                entry = {
                    'name': '',
                    'symbol': symbol,
                    'mint': mint,
                    'decimals': 9,
                    'token_program': token_prog,
                    'logo_uri': '',
                    'pool': {
                        'program': '',
                        'pool': pool_address,
                        'validator_list': ''
                    }
                }
                sanctum_lst_list.append(entry)
        
        # Create the data structure with header comment
        data = {
            'sanctum_lst_list': sanctum_lst_list
        }
        
        with open(self.results_file, 'w') as f:
            f.write("# SanctumSpl Pool -- S upgrade path\n\n")
            toml.dump(data, f)
        
        # Also save backup in original format for resuming
        backup_data = {
            'pools': self.stable_usdc_pools,
            'total_count': len(self.stable_usdc_pools),
            'timestamp': time.time()
        }
        backup_file = self.results_file.replace('.toml', '_backup.toml')
        with open(backup_file, 'w') as f:
            toml.dump(backup_data, f)
    
    def fetch_meteora_pools(self, query_params=None):
        if query_params is None:
            query_params = {
                "page": 0,
                "size": 100,
                "pool_type": "dynamic",
                "sort_key": "tvl",
                "order_by": "desc"
            }
        
        headers = {
            'accept': '*/*',
            'content-type': 'text/plain;charset=UTF-8',
        }

        params = [(f"{key}={value}") for key, value in query_params.items()]
        query_string = "&".join(params)
        url = f"https://amm-v2.meteora.ag/pools/search?{query_string}"
        
        payload = json.dumps({"url": url})
        
        api_endpoint = 'https://www.meteora.ag/api/fetch'
        return requests.post(api_endpoint, headers=headers, data=payload)

    def collect_pools(self, max_pages=100, page_size=100, save_every=10):
        usdc_token_address = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
        consecutive_failures = 0
        
        for page_number in range(max_pages):
            if self.interrupted:
                break
                
            query_params = {
                "page": page_number,
                "size": page_size,
                "pool_type": "dynamic",
                "sort_key": "tvl",
                "order_by": "desc"
            }
            
            try:
                response = self.fetch_meteora_pools(query_params)
                
                if response.status_code == 200:
                    consecutive_failures = 0
                    response_data = response.json()
                    
                    if 'data' not in response_data:
                        continue
                    
                    pools = response_data['data']
                    
                    if not pools:
                        break
                    
                    new_pools = 0
                    for pool in pools:
                        token_mints = pool.get('pool_token_mints', [])
                        is_stable_pool = pool.get('pool_type') == 'stable'
                        contains_usdc = usdc_token_address in token_mints
                        pool_id = pool.get('pool_id')
                        
                        if not (is_stable_pool and contains_usdc):
                            continue
                        
                        if pool_id:
                            if pool_id not in self.pool_ids:
                                self.stable_usdc_pools.append(pool)
                                self.pool_ids.add(pool_id)
                                new_pools += 1
                        else:
                            self.stable_usdc_pools.append(pool)
                            new_pools += 1
                    
                    total = len(self.stable_usdc_pools)
                    page_num = page_number + 1
                    print(f"Page {page_num}: {new_pools} new pools (total: {total})")
                    
                    if (page_number + 1) % save_every == 0:
                        self.save_results()
                        print(f"Progress saved at page {page_number + 1}")
                    
                else:
                    consecutive_failures += 1
                    if consecutive_failures >= 5:
                        break
                    
            except Exception:
                consecutive_failures += 1
                if consecutive_failures >= 5:
                    break
                    
            time.sleep(2.0)
        
        self.save_results()
        return self.stable_usdc_pools

    def test_api_connection(self):
        response = self.fetch_meteora_pools()
        return response.status_code == 200

    def display_results(self):
        print(f"\nTotal pools found: {len(self.stable_usdc_pools)}")
        
        for i, pool in enumerate(self.stable_usdc_pools, 1):
            pool_id = pool.get('pool_id', 'N/A')
            tvl = pool.get('tvl', 0)
            token_mints = pool.get('pool_token_mints', [])
            pool_type = pool.get('pool_type', 'N/A')
            
            print(f"\nPool {i}:")
            print(f"Pool ID: {pool_id}")
            print(f"TVL: ${tvl:,.2f}" if tvl else "TVL: N/A")
            print(f"Token Mints: {token_mints}")
            print(f"Pool Type: {pool_type}")
            print("-" * 50)


if __name__ == "__main__":
    collector = StableUSDCPoolCollector()
    
    if collector.test_api_connection():
        collector.collect_pools(max_pages=1000, save_every=1)
        collector.display_results()
    else:
        print("API connection failed")

Page 1: 1 new pools (total: 1)
Progress saved at page 1
Page 2: 1 new pools (total: 2)
Progress saved at page 2
Page 3: 1 new pools (total: 3)
Progress saved at page 3
Page 4: 0 new pools (total: 3)
Progress saved at page 4
Page 5: 0 new pools (total: 3)
Progress saved at page 5
Page 6: 0 new pools (total: 3)
Progress saved at page 6

Interrupted! Saving 3 pools...
Results saved to stable_usdc_pools.toml

Total pools found: 3

Pool 1:
Pool ID: N/A
TVL: N/A
Token Mints: ['EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', 'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB']
Pool Type: stable
--------------------------------------------------

Pool 2:
Pool ID: N/A
TVL: N/A
Token Mints: ['Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB', 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v']
Pool Type: stable
--------------------------------------------------

Pool 3:
Pool ID: N/A
TVL: N/A
Token Mints: ['EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', '7kbnvuGBxxj8AG9qp8Scn56muWGaRaFqxg1FsRp3PaFT']
Pool Type

SystemExit: 0

/home/analytics/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
